In [20]:
import pandas as pd
import numpy as np
from copy import copy

In [21]:
pd.set_option('mode.chained_assignment', None)

In [22]:
#crsp_raw = pd.read_csv('CRSP stock price421.csv')
crsp_raw = pd.read_csv('stockdata.csv')
crsp_raw = crsp_raw.drop(columns = 'Unnamed: 0')

In [23]:
compustat_raw = pd.read_csv('fundamentals2010c.csv')

In [24]:
def clean_data(df, type_dict):
    print('Cleaning date variables:')
    for v in type_dict['date_vars']:
        print(v)
        df[v] = pd.to_datetime(df[v], format = '%Y%m%d', errors = 'coerce')
        
    print('Cleaning numeric variables:')
    for v in type_dict['float_vars']:
        print(v)
        df[v] = pd.to_numeric(df[v], errors = 'coerce')
    
    print('Cleaning integer variables:')
    for v in type_dict['int_vars']:
        print(v)
        df[v] = pd.to_numeric(df[v], downcast = 'signed', errors = 'coerce')
        
    print('Final data types:')
    print(df.dtypes)
    
    return df

In [25]:
crsp_datatypes = {'date_vars': ['date'],
                 'float_vars': ['PRC', 'DIVAMT', 'BID', 'ASK', 'CFACPR', 'CFACSHR', "RET"],
                 'int_vars': ['SHROUT', 'VOL']}
crsp = clean_data(copy(crsp_raw), crsp_datatypes)

Cleaning date variables:
date
Cleaning numeric variables:
PRC
DIVAMT
BID
ASK
CFACPR
CFACSHR
RET
Cleaning integer variables:
SHROUT
VOL
Final data types:
PERMNO              int64
date       datetime64[ns]
NCUSIP             object
TICKER             object
COMNAM             object
DIVAMT            float64
PRC               float64
VOL               float64
RET               float64
BID               float64
ASK               float64
SHROUT            float64
CFACPR            float64
CFACSHR           float64
dtype: object


In [14]:
# Choose the right variables
crsp_names = {
              "RET" : "Return",
              'SHROUT': 'Shares Outstanding on Trading Day', 
              'COMNAM': 'Company Name',\
              'date': 'datadate', 
              'NCUSIP': 'cusip',
              "TICKER": "Ticker",
              'DIVAMT': 'Dividend Cash Amount',
              'PRC': 'Price',
              'BID': 'Bid',
              'ASK': 'Ask',
              'VOL': 'Volume on Trading Day',
              'CFACPR': 'Price Adjustment Factor',
              'CFACSHR': 'Share Adjustment Factor'}

crsp = crsp.rename(index = str, columns = crsp_names)
crsp = crsp[list(crsp_names.values())]

In [15]:
# Make a few more useful variables
crsp['Price'] = np.abs(crsp['Price'])
crsp['Shares Outstanding'] = crsp['Shares Outstanding on Trading Day'] * crsp['Share Adjustment Factor']
crsp['Volume'] = crsp['Volume on Trading Day'] * crsp['Share Adjustment Factor']
crsp['Market Cap'] = crsp['Shares Outstanding'] * crsp['Price'] / 1e6   #million

In [16]:
# Drop a few variables
crsp = crsp.drop(['Shares Outstanding on Trading Day', 'Volume on Trading Day', 'Bid', 'Ask', "Volume", "Price Adjustment Factor", "Share Adjustment Factor"], axis = 1)

In [17]:
# Check for unique identifier

def check_unique(dataframe, identifier_list):
    unique_identifier = dataframe.groupby(by = identifier_list).count()
    return unique_identifier.shape[0] == dataframe.shape[0]

print(check_unique(crsp, ['cusip', 'datadate']))

False


In [19]:

crsp

,Return,Company Name,datadate,cusip,Ticker,Dividend Cash Amount,Price,Shares Outstanding,Market Cap
0,0.102144,ORACLE CORP,NaT,68389X10,ORCL,0.06,28.21000,5025837.0,141.778862
1,0.037044,ORACLE CORP,NaT,68389X10,ORCL,NaN,29.25500,4979000.0,145.660645
2,-0.003247,ORACLE CORP,NaT,68389X10,ORCL,NaN,29.16000,4975106.0,145.074091
3,0.010288,ORACLE CORP,NaT,68389X10,ORCL,0.06,29.40000,4975106.0,146.268116
4,-0.099660,ORACLE CORP,NaT,68389X10,ORCL,NaN,26.47000,4905000.0,129.835350
...,...,...,...,...,...,...,...,...,...
48868,-0.066222,TESLA INC,NaT,88160R10,TSLA,NaN,225.61000,895635.0,202.064212
48869,0.067639,TESLA INC,NaT,88160R10,TSLA,NaN,240.87000,900000.0,216.783000
48870,0.307427,TESLA INC,NaT,88160R10,TSLA,NaN,314.92001,901225.0,283.813786
48871,0.047695,TESLA INC,NaT,88160R10,TSLA,NaN,329.94000,901225.0,297.350176


In [27]:

price = crsp[['date', 'TICKER', 'PRC', 'SHROUT', 'VOL']].copy()

In [28]:
price

,date,TICKER,PRC,SHROUT,VOL
0,NaT,ORCL,28.21000,5025837.0,7248771.0
1,NaT,ORCL,29.25500,4979000.0,5679684.0
2,NaT,ORCL,29.16000,4975106.0,8170098.0
3,NaT,ORCL,29.40000,4975106.0,5364162.0
4,NaT,ORCL,26.47000,4905000.0,6501551.0
...,...,...,...,...,...
48868,NaT,TSLA,225.61000,179127.0,1340932.0
48869,NaT,TSLA,240.87000,180000.0,1365915.0
48870,NaT,TSLA,314.92001,180245.0,2351125.0
48871,NaT,TSLA,329.94000,180245.0,1578851.0
